### Course Recommendation System using Udemy Dataset¶
#### Algo¶
    Cosine Similarity
    Linear Similarity
#### Workflow¶
    Dataset
    Vectorized our dataset
    Cosine Similarity Matrix
    ID,Score
    Recommend

Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import neattext.functions as nfx
import seaborn as sn

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [2]:
df = pd.read_csv('udemy_courses.csv')
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance


Neattext library for nlp 

In [3]:
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [4]:
df.course_title.iloc[1:5]

1    Complete GST Course & Certification - Grow You...
2    Financial Modeling for Business Analysts and C...
3    Beginner to Pro - Financial Analysis in Excel ...
4         How To Maximize Your Profits Trading Options
Name: course_title, dtype: object

### Data Preprocessing
1. Remove Stopwords
2. Remove Special Characters
3. Vectorize the Course title

In [5]:
df['Clean_title'] = df.course_title.apply(nfx.remove_stopwords)

In [6]:
df.Clean_title = df.Clean_title.apply(nfx.remove_punctuations)

In [8]:
df.Clean_title.iloc[1:8]

1    Complete GST Course  Certification  Grow Practice
2     Financial Modeling Business Analysts Consultants
3          Beginner Pro  Financial Analysis Excel 2017
4                     Maximize Profits Trading Options
5              Trading Penny Stocks: Guide Levels 2017
6    Investing Trading Beginners: Mastering Price C...
7    Trading Stock Chart Patterns Immediate Explosi...
Name: Clean_title, dtype: object

In [9]:
# vectorizing the course_title

countvect = CountVectorizer()

cv_mat = countvect.fit_transform(df['Clean_title'])

cv_mat

<3678x3684 sparse matrix of type '<class 'numpy.int64'>'
	with 18548 stored elements in Compressed Sparse Row format>

In [11]:
cv_mat.todense().shape

(3678, 3684)

In [12]:
df_cv_words = pd.DataFrame(cv_mat.todense(), 
                           columns=countvect.get_feature_names())

df_cv_words

C:\Users\SIDDHANT\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000005,001,01,02,06z,08,10,100,101,101master,...,超絶技巧,青色申告の記帳を自力で行うための複式簿記の基本,만들기,캔바,콘텐츠,１日,１日でマスター,１週間でホームページのトップ画像を製作できるようになる画像製作講座,６時間でインターバンク市場を攻略,７日でマスター
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
cosine_sim_mat = cosine_similarity(cv_mat)

cosine_sim_mat

array([[1.        , 0.20412415, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20412415, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.23570226],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.23570226, 0.        ,
        1.        ]])

In [15]:
course_index = pd.Series(df.index,index = df['course_title']).drop_duplicates()
course_index


course_title
Ultimate Investment Banking Course                                0
Complete GST Course & Certification - Grow Your CA Practice       1
Financial Modeling for Business Analysts and Consultants          2
Beginner to Pro - Financial Analysis in Excel 2017                3
How To Maximize Your Profits Trading Options                      4
                                                               ... 
Learn jQuery from Scratch - Master of JavaScript library       3673
How To Design A WordPress Website With No Coding At All        3674
Learn and Build using Polymer                                  3675
CSS Animations: Create Amazing Effects on Your Website         3676
Using MODX CMS to Build Websites: A Beginner's Guide           3677
Length: 3678, dtype: int64

In [21]:
course_dict = dict(course_index)


course_keys = list(course_dict.keys())

touse = {}
counter = 0
keyword = 'Java'
for key,value in course_dict.items():
    if keyword in key:
        touse[counter] = key
        
    counter+=1
    
touse

{2468: 'Learning Dynamic Website Design - PHP MySQL and JavaScript',
 2479: 'Learn JavaScript for beginners',
 2493: 'Accelerated JavaScript Training',
 2501: 'Advanced Javascript',
 2510: 'Start Writing JavaScript Today - Beginner JavaScript Course',
 2515: 'Javascript ES6! A Complete Reference Guide to Javascript ES6',
 2521: 'Javascript Intermediate level 2 - Mastering Functions',
 2525: 'Learn JavaScript for Web Development',
 2531: 'Javascript Intermediate level 3 - JSON',
 2534: 'JavaScript DOM Dynamic Web interactive content Boot Camp',
 2535: 'Master JavaScript Programming, 3 Projects Included !!',
 2543: 'Dynamic JavaScript Master Class AJAX JSON Simple APIs',
 2546: 'JavaScript the Basics - JavaScript for Beginners',
 2547: 'Complete E-Commerce Course - Java,Spring,Hibernate and MySQL',
 2548: 'Learn SVG Animation - With HTML, CSS & Javascript',
 2575: 'JavaScript For Absolute Beginners - Level One',
 2588: 'Master ExpressJS to Build Web Apps with NodeJS&JavaScript',
 2607: '

In [22]:
temp = df[df['course_title'].str.contains('Python')]
temp.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Clean_title
14,1196544,Python Algo Trading: Sentiment Trading with News,https://www.udemy.com/hedge-fund-strategy-trad...,True,200,294,19,42,All Levels,7.0,2017-04-28T16:41:44Z,Business Finance,Python Algo Trading: Sentiment Trading News
30,1170894,Python Algo Stock Trading: Automate Your Trading!,https://www.udemy.com/algorithmic-stock-tradin...,True,95,1165,21,41,Beginner Level,2.5,2017-05-28T23:41:03Z,Business Finance,Python Algo Stock Trading: Automate Trading
41,1035472,Python for Finance: Investment Fundamentals & ...,https://www.udemy.com/python-for-finance-inves...,True,195,3811,278,103,All Levels,6.5,2017-03-30T22:17:09Z,Business Finance,Python Finance: Investment Fundamentals Data ...
147,1070886,Python Algo Trading: FX Trading with Oanda,https://www.udemy.com/python-algo-trading-fx-t...,True,200,453,42,33,Intermediate Level,3.0,2017-03-14T00:39:45Z,Business Finance,Python Algo Trading: FX Trading Oanda
334,815482,Stock Technical Analysis with Python,https://www.udemy.com/stock-technical-analysis...,True,50,409,35,46,All Levels,8.0,2016-04-12T00:40:03Z,Business Finance,Stock Technical Analysis Python


In [23]:
top6 = temp.sort_values(by = 'num_subscribers',ascending=False).head(6)

top6

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Clean_title
2497,16646,Web Programming with Python,https://www.udemy.com/web-programming-with-pyt...,True,50,35267,217,53,All Levels,4.0,2012-04-25T00:01:43Z,Web Development,Web Programming Python
2570,47963,"Coding for Entrepreneurs: Learn Python, Django...",https://www.udemy.com/coding-for-entrepreneurs/,True,195,23412,799,251,All Levels,45.0,2013-04-08T00:46:14Z,Web Development,Coding Entrepreneurs: Learn Python Django More
2528,391546,Learn Python and Django: Payment Processing,https://www.udemy.com/learn-django-code-accept...,True,70,17714,198,23,All Levels,3.5,2015-02-09T15:37:56Z,Web Development,Learn Python Django: Payment Processing
3200,822444,Python and Django Full Stack Web Developer Boo...,https://www.udemy.com/python-and-django-full-s...,True,200,11832,1883,191,All Levels,31.5,2017-02-24T18:40:55Z,Web Development,Python Django Stack Web Developer Bootcamp
3507,394832,Fun and creative web engineering with Python a...,https://www.udemy.com/web-engineering-with-pyt...,False,0,10917,319,25,All Levels,2.0,2015-06-09T19:51:50Z,Web Development,Fun creative web engineering Python Web2py
2553,938560,The Complete Ethical Hacking Course 2.0: Pytho...,https://www.udemy.com/penetration-testing-ethi...,True,195,7827,268,66,All Levels,11.0,2016-09-26T15:08:29Z,Web Development,Complete Ethical Hacking Course 20: Python Ka...


In [24]:
index = course_index['How To Maximize Your Profits Trading Options']


In [28]:
scores = list(enumerate(cosine_sim_mat[index]))
scores[1:20]


[(1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 1.0),
 (5, 0.20412414523193154),
 (6, 0.20412414523193154),
 (7, 0.1889822365046136),
 (8, 0.3779644730092272),
 (9, 0.0),
 (10, 0.20412414523193154),
 (11, 0.5),
 (12, 0.0),
 (13, 0.17677669529663687),
 (14, 0.35355339059327373),
 (15, 0.0),
 (16, 0.0),
 (17, 0.1889822365046136),
 (18, 0.22360679774997896),
 (19, 0.0)]

In [29]:
sorted_score = sorted(scores,key = lambda x:x[1],reverse=True)
sorted_score[1:20]

[(408, 0.5773502691896258),
 (43, 0.5669467095138407),
 (94, 0.5303300858899106),
 (136, 0.5303300858899106),
 (193, 0.5303300858899106),
 (442, 0.5303300858899106),
 (801, 0.5303300858899106),
 (11, 0.5),
 (59, 0.5),
 (68, 0.5),
 (71, 0.5),
 (95, 0.5),
 (328, 0.5),
 (376, 0.5),
 (512, 0.5),
 (645, 0.5),
 (736, 0.5),
 (944, 0.5),
 (987, 0.5)]

In [31]:
# so the sorted score list is a collection of tuples 
# which have the index and the value,so i will select the indices first

sorted_indices = [i[0] for i in sorted_score[1:]]

sorted_values = [i[1] for i in sorted_score[1:]]

sorted_values[0:20]

[0.5773502691896258,
 0.5669467095138407,
 0.5303300858899106,
 0.5303300858899106,
 0.5303300858899106,
 0.5303300858899106,
 0.5303300858899106,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.45226701686664544]

In [32]:
recommended_result_df = df.iloc[sorted_indices]

recommended_result_df

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Clean_title
408,889066,Trading Options Basics,https://www.udemy.com/trading-options-basics/,True,200,8,0,8,Beginner Level,1.5,2016-07-01T03:13:22Z,Business Finance,Trading Options Basics
43,627540,Options Trading - How to Win with Weekly Options,https://www.udemy.com/work-from-home-setup-you...,True,115,7489,1190,25,Intermediate Level,1.0,2015-10-22T21:54:28Z,Business Finance,Options Trading Win Weekly Options
94,474928,Intermediate Options trading concepts for Stoc...,https://www.udemy.com/intermediate-options-tra...,True,40,2000,30,9,All Levels,1.0,2015-04-13T20:28:04Z,Business Finance,Intermediate Options trading concepts Stocks O...
136,1243448,Forex Trading with Fixed 'Risk through Options...,https://www.udemy.com/forexoptions/,True,200,611,4,26,Beginner Level,1.0,2017-06-07T17:15:07Z,Business Finance,Forex Trading Fixed Risk Options Trading
193,919906,Trading Options For Consistent Returns: Option...,https://www.udemy.com/trading-options-for-income/,False,0,4077,281,20,Beginner Level,1.5,2016-08-18T21:57:04Z,Business Finance,Trading Options Consistent Returns: Options Ba...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3673,775618,Learn jQuery from Scratch - Master of JavaScri...,https://www.udemy.com/easy-jquery-for-beginner...,True,100,1040,14,21,All Levels,2.0,2016-06-14T17:36:46Z,Web Development,Learn jQuery Scratch Master JavaScript library
3674,1088178,How To Design A WordPress Website With No Codi...,https://www.udemy.com/how-to-make-a-wordpress-...,True,25,306,3,42,Beginner Level,3.5,2017-03-10T22:24:30Z,Web Development,Design WordPress Website Coding
3675,635248,Learn and Build using Polymer,https://www.udemy.com/learn-and-build-using-po...,True,40,513,169,48,All Levels,3.5,2015-12-30T16:41:42Z,Web Development,Learn Build Polymer
3676,905096,CSS Animations: Create Amazing Effects on Your...,https://www.udemy.com/css-animations-create-am...,True,50,300,31,38,All Levels,3.0,2016-08-11T19:06:15Z,Web Development,CSS Animations: Create Amazing Effects Website


In [33]:
recommended_result_df['Similarity_Score'] = np.array(sorted_values)

recommended_result_df

C:\Users\SIDDHANT\AppData\Local\Temp/ipykernel_2068/1637388502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_result_df['Similarity_Score'] = np.array(sorted_values)


,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Clean_title,Similarity_Score
408,889066,Trading Options Basics,https://www.udemy.com/trading-options-basics/,True,200,8,0,8,Beginner Level,1.5,2016-07-01T03:13:22Z,Business Finance,Trading Options Basics,0.577350
43,627540,Options Trading - How to Win with Weekly Options,https://www.udemy.com/work-from-home-setup-you...,True,115,7489,1190,25,Intermediate Level,1.0,2015-10-22T21:54:28Z,Business Finance,Options Trading Win Weekly Options,0.566947
94,474928,Intermediate Options trading concepts for Stoc...,https://www.udemy.com/intermediate-options-tra...,True,40,2000,30,9,All Levels,1.0,2015-04-13T20:28:04Z,Business Finance,Intermediate Options trading concepts Stocks O...,0.530330
136,1243448,Forex Trading with Fixed 'Risk through Options...,https://www.udemy.com/forexoptions/,True,200,611,4,26,Beginner Level,1.0,2017-06-07T17:15:07Z,Business Finance,Forex Trading Fixed Risk Options Trading,0.530330
193,919906,Trading Options For Consistent Returns: Option...,https://www.udemy.com/trading-options-for-income/,False,0,4077,281,20,Beginner Level,1.5,2016-08-18T21:57:04Z,Business Finance,Trading Options Consistent Returns: Options Ba...,0.530330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3673,775618,Learn jQuery from Scratch - Master of JavaScri...,https://www.udemy.com/easy-jquery-for-beginner...,True,100,1040,14,21,All Levels,2.0,2016-06-14T17:36:46Z,Web Development,Learn jQuery Scratch Master JavaScript library,0.000000
3674,1088178,How To Design A WordPress Website With No Codi...,https://www.udemy.com/how-to-make-a-wordpress-...,True,25,306,3,42,Beginner Level,3.5,2017-03-10T22:24:30Z,Web Development,Design WordPress Website Coding,0.000000
3675,635248,Learn and Build using Polymer,https://www.udemy.com/learn-and-build-using-po...,True,40,513,169,48,All Levels,3.5,2015-12-30T16:41:42Z,Web Development,Learn Build Polymer,0.000000
3676,905096,CSS Animations: Create Amazing Effects on Your...,https://www.udemy.com/css-animations-create-am...,True,50,300,31,38,All Levels,3.0,2016-08-11T19:06:15Z,Web Development,CSS Animations: Create Amazing Effects Website,0.000000


In [34]:
use_df = recommended_result_df[['Clean_title','Similarity_Score']]
use_df


,Clean_title,Similarity_Score
408,Trading Options Basics,0.577350
43,Options Trading Win Weekly Options,0.566947
94,Intermediate Options trading concepts Stocks O...,0.530330
136,Forex Trading Fixed Risk Options Trading,0.530330
193,Trading Options Consistent Returns: Options Ba...,0.530330
...,...,...
3673,Learn jQuery Scratch Master JavaScript library,0.000000
3674,Design WordPress Website Coding,0.000000
3675,Learn Build Polymer,0.000000
3676,CSS Animations: Create Amazing Effects Website,0.000000


In [36]:
def recommend_course(title,numrec = 10):
    
    course_index = pd.Series(
        df.index, index=df['course_title']).drop_duplicates()

    index = course_index[title]

    scores = list(enumerate(cosine_sim_mat [index]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    selected_course_index = [i[0] for i in sorted_scores[1:]]

    selected_course_score = [i[1] for i in sorted_scores[1:]]

    rec_df = df.iloc[selected_course_index]

    rec_df['Similarity_Score'] = selected_course_score

    final_recommended_courses = rec_df[[
        'course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']]

    return final_recommended_courses.head(numrec)

In [41]:
ans = recommend_course('Trading Options Basics',20)

ans

C:\Users\SIDDHANT\AppData\Local\Temp/ipykernel_2068/2062375757.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_df['Similarity_Score'] = selected_course_score


,course_title,Similarity_Score,url,price,num_subscribers
95,Options Trading 101: The Basics,0.866025,https://www.udemy.com/options-trading-101-the-...,0,1514
193,Trading Options For Consistent Returns: Option...,0.816497,https://www.udemy.com/trading-options-for-income/,0,4077
861,Basics of Trading,0.816497,https://www.udemy.com/basicsoftrading/,40,99
66,Options Trading Basics (3-Course Bundle),0.774597,https://www.udemy.com/learn-options-trading-co...,180,10100
800,Trading: Basics of Trading for Beginners,0.707107,https://www.udemy.com/money-basics-of-trading-...,140,1309
953,Options Basics & Trading With Small Capital! -...,0.707107,https://www.udemy.com/options-basics-trading-w...,30,120
43,Options Trading - How to Win with Weekly Options,0.654654,https://www.udemy.com/work-from-home-setup-you...,115,7489
94,Intermediate Options trading concepts for Stoc...,0.612372,https://www.udemy.com/intermediate-options-tra...,40,2000
136,Forex Trading with Fixed 'Risk through Options...,0.612372,https://www.udemy.com/forexoptions/,200,611
442,The Advantages of ETF Options and Index Option...,0.612372,https://www.udemy.com/learn-etf-options-and-in...,60,52


In [42]:
ans.shape

(20, 5)

In [43]:
ans.columns

Index(['course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers'], dtype='object')

In [44]:
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Clean_title
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance,Ultimate Investment Banking Course
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance,Complete GST Course Certification Grow Practice
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance,Financial Modeling Business Analysts Consultants
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance,Beginner Pro Financial Analysis Excel 2017
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance,Maximize Profits Trading Options


In [45]:
df.to_csv('UdemyCleanedTitle.csv',index = None)